<a href="https://colab.research.google.com/github/nafisulhaque98/id3-decision-tree-titanic/blob/main/ID3_Decision_Tree_(13423126%2C_Assignment_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Creating the nodes for the tree structure
class TreeNode:
  def __init__(self, name, value, attribute):
    self.attribute = attribute
    self.name = name
    self.value = value # data
    self.children = [] # references to other nodes
    self.result = None
    self.parent = None
  
  def set_result(self,new_result):
    self.result = new_result

  def set_parent(self, parent_node):
    self.parent = parent_node    

  def add_child(self, child_node):
    # creates parent-child relationship
    # print("Adding " + child_node.value)
    self.children.append(child_node)
    
  def remove_child(self, child_node):
    # removes parent-child relationship
    print("Removing " + child_node.value + " from " + self.value)
    self.children = [child for child in self.children 
                     if child is not child_node]

  def traverse(self):
    # moves through each node referenced from self downwards
    nodes_to_visit = [self]
    while len(nodes_to_visit) > 0:
      current_node = nodes_to_visit.pop()
      print(current_node.value)
      nodes_to_visit += current_node.children



In [ ]:
#Mounting Google Drive to import Training dataset
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Reading training dataset
import pandas as pd
df = pd.read_csv("gdrive/My Drive/DecisionTree/train trans part.csv")
df

,Survived,Pclass,Sex,Age,Embarked,FamilySize
0,No,3,male,Young adult,S,1
1,Yes,3,female,Young adult,S,0
2,No,3,male,Middle-aged,S,0
3,No,1,male,Senior,S,0
4,No,3,male,Child,S,4
...,...,...,...,...,...,...
600,No,2,male,Young adult,S,0
601,No,3,female,Middle-aged,Q,5
602,Yes,1,female,Teen,S,0
603,Yes,1,male,Young adult,C,0


In [ ]:
import math
from math import log
def make_decision_tree(df, target_column):
  
  # Compute entropy H(C) 
  def entropy():
    h=0
    for feature in df[target_column].value_counts():
      probability = feature/len(df)
      h += probability*math.log(probability,2)
    h*=-1
    return h

  def info_gain_v2(df,target_column,column):
    #compute the P(column)
    
    probabilities = df.groupby(column).size().div(len(df))
    H_value = 0
    for index1, value1 in probabilities.iteritems():
      tmp_df = df[df[column] == index1]
      tmp_probability = tmp_df.groupby(target_column).size().div(len(tmp_df))
      tmp_v =0 
      # compute H(C|X=Xi)
      for v2 in tmp_probability:
        tmp_v += -v2 * log(v2,2)
    
      #compute H(y_yol| column)
      H_value += value1 * tmp_v

    return H_value 

  def get_max_info_gain_v2(df, target_column):

    h_value = entropy()
    info_gains ={}
    for column in filter(lambda column: column != target_column, df.columns):
      #compute H(y-col) - H(y_col|column)
      info_gains[column] = h_value - info_gain_v2(df,target_column,column)
    
    max_info_gain = max(info_gains,key=info_gains.get)

    return max_info_gain

  def train_tree(node,df,target_column):
    # which attribute to go now, attribute with the max_info_gain
    current_column = get_max_info_gain_v2(df,target_column)
    # create nodes for each feature in this attribute for feature in pd.unique(df[current_column])  
    for feature in pd.unique(df[current_column]):
      name = '%s-%s' % (current_column,feature)
      current_node = TreeNode(name, feature, current_column)
      # connect these nodes with parent node
      current_node.set_parent(node)
      node.add_child(current_node)  

      gb = df[df[current_column] == feature].groupby(target_column)
      # if we have more than two attributes
      if len(df.columns) > 2 : 
        # check if this one is the leaf node
        if len(gb) == 1:
          name = df[df[current_column] == feature].groupby(current_column)[target_column].first().iloc[0]
          leafnode = TreeNode(name,None,None)
          leafnode.set_result(name)
          leafnode.set_parent(current_node)
          current_node.add_child(leafnode)
        else:
          # call this method again with new df, where the current attribute has been removed 
          train_tree(current_node,df[df[current_column] == feature].drop(current_column,axis=1),target_column)
    # if attributes runs out, we pick the most frequent value as the leaf node
      else:  
        name = df[df[current_column] == feature].groupby(target_column).size().idxmax()
        leafnode = TreeNode(name,None,None)
        leafnode.set_result(name)
        leafnode.set_parent(current_node)
        current_node.add_child(leafnode)


  root_node = TreeNode('root',None,None)
  train_tree(root_node,df,target_column)
  return root_node

In [ ]:
#To show the tree structure built by the model and the order of the attributes it is reading
!pip install pptree

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pptree import *

root_nood=make_decision_tree(df, "Survived")

print_tree(root_nood, childattr='children', nameattr='name', horizontal=True)

                         ┌FamilySize-1┐
                         │            └Yes
                         ├FamilySize-5┐
                         │            └Yes
                         ├FamilySize-2┐
                         │            └Yes
                         ├FamilySize-4┐
                         │            └Yes
                ┌Pclass-1┤
                │        │            ┌Age-Senior┐
                │        │            │          └Yes
                │        │            ├Age-Young adult┐
                │        │            │               └Yes
                │        ├FamilySize-0┤
                │        │            │               ┌Embarked-C┐
                │        │            │               │          └Yes
                │        │            ├Age-Middle-aged┤
                │        │            │               └Embarked-S┐
                │        │            │                          └Yes
                │        │            └Age-Teen┐
  

In [ ]:
#Creating the Prediction for the dataset

def make_decision(node, sample):
    if node.result is not None:
      return node.result
    else:
      for child in node.children:
        if child.result is not None:
          return child.result
        else:
          if child.value == sample[child.attribute]:
            return make_decision(child, sample)

In [ ]:
#Importing the testing dataset
import pandas as pd
test_df = pd.read_csv("gdrive/My Drive/DecisionTree/test trans part.csv")
test_df

,Survived,Pclass,Sex,Age,Embarked,FamilySize
0,Yes,1,female,Middle-aged,C,1
1,Yes,1,female,Middle-aged,S,1
2,Yes,3,female,Young adult,S,2
3,Yes,2,female,Young adult,S,1
4,No,3,female,Teen,S,7
...,...,...,...,...,...,...
102,No,3,male,Middle-aged,S,0
103,Yes,2,female,Young adult,C,1
104,No,3,male,Teen,S,0
105,No,3,male,Young adult,S,0


In [ ]:
def evaluation(node, samples, target_column):
  correct = 0
  i = 0

  while i < len(samples):
    sample = samples.iloc[i]
    result = make_decision(node, sample)
    i = i + 1
    if result == sample[target_column]:
      correct = correct + 1
      print("Correct")
    else:
      print("Wrong")
    
  accuracy = correct / len(samples) * 100 
  print("Sample number =", len(samples), "Correct prediction percentage =", accuracy, "%")


In [ ]:

evaluation(root_nood, test_df, "Survived")

Correct
Correct
Wrong
Correct
Correct
Wrong
Correct
Wrong
Correct
Correct
Correct
Correct
Correct
Wrong
Wrong
Correct
Correct
Correct
Correct
Correct
Correct
Wrong
Correct
Correct
Correct
Correct
Correct
Correct
Wrong
Correct
Correct
Wrong
Correct
Wrong
Wrong
Wrong
Wrong
Correct
Wrong
Correct
Wrong
Correct
Wrong
Wrong
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Wrong
Correct
Wrong
Correct
Correct
Correct
Correct
Wrong
Correct
Correct
Correct
Correct
Wrong
Correct
Correct
Correct
Correct
Correct
Correct
Wrong
Wrong
Correct
Correct
Wrong
Correct
Correct
Wrong
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Wrong
Correct
Correct
Correct
Wrong
Wrong
Correct
Correct
Wrong
Correct
Correct
Wrong
Correct
Correct
Correct
Wrong
Correct
Correct
Correct
Sample number = 107 Correct prediction percentage = 71.96261682242991 %
